In [ ]:
import numpy as np
import scipy as sp
from PIL import Image
import taichi as ti
ti.init(arch=ti.cpu, debug=False)
import eikivp
from eikivp.M2.vesselness import multi_scale_vesselness, export_vesselness
from eikivp.utils import cost_function
import matplotlib.pyplot as plt

# Parameters

In [2]:
σ_s_list = np.array((0.5**3, 0.5)) # np.array((1.5, 2.))
σ_o = 0.5 * 0.75**2
σ_s_ext = 1.
σ_o_ext = 0.01
λ = 100
p = 2
image_name = "E46_OD_best"
image_file_name = f"data\{image_name}.tif"

params = {
    "σ_s_list": σ_s_list,
    "σ_o": σ_o,
    "σ_s_ext": σ_s_ext,
    "σ_o_ext": σ_o_ext,
    "image_name": image_name 
}

# Preprocess Image

In [ ]:
ds = 8
retinal_image = Image.open(image_file_name)
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_unnormalised = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array = eikivp.R2.utils.align_to_real_axis_scalar_field(eikivp.utils.image_rescale(retinal_array_unnormalised)) # So that +x is in the same direction as +I and +y is in the same direction as +J
u = retinal_array - sp.ndimage.gaussian_filter(retinal_array, 16, truncate=2., mode="nearest")
# We need to rescale, but also in Mathematica imHP is no longer in [0, 1].
eikivp.visualisations.view_image_array((u - u.min()) / (u.max() - u.min()));

In [4]:
dim_I, dim_J = retinal_array.shape
dim_K = 32
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
dxy = 1.
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2. * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = eikivp.M2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)
bifurcations_list = (
    (58, 175),  (69, 184),  (141, 212), (135, 201), (149, 182), (124,153), 
    (110, 126), (86, 84),   (92, 91),   (143, 74),  (170, 129), (177,190), 
    (191, 276), (213, 296), (195, 298), (167, 285), (176, 259), (131, 294),
    (95, 290),  (62, 229),  (97, 242),  (176, 303), (116, 329), (34, 352), 
    (41, 342),  (195, 178), (185, 120), (198, 32),  (219, 97),  (240, 84), 
    (232, 92),  (245, 127), (242, 128), (233, 133), (201, 185), (209, 186), 
    (186, 214), (189, 234), (232, 290), (184, 74),  (172, 46),  (320, 150), 
    (318, 169), (304, 169), (314, 219), (286, 192), (335,232),  (326, 181), 
    (343, 208), (335, 252), (307, 284), (345, 161), (355, 118), (373, 173), 
    (414, 133), (339, 278), (408, 335), (255,290),  (322, 375), (322, 421), 
    (297, 365), (262, 320), (268, 464), (290, 453), (337, 370), (310, 342), 
    (315, 342), (248, 345), (221,331),  (210, 338), (183, 376), (185, 341), 
    (337, 161), (384, 93),  (241, 318), (49, 312),  (49, 311),  (126, 207), 
    (142, 134), (159, 89),  (112, 58),  (279, 24),  (303, 129), (298, 132), 
    (273, 176), (269, 186), (284, 209), (287, 219), (272, 241), (282, 283), 
    (297, 275), (379,234),  (358, 246), (309, 48),  (268, 313), (326, 266)
)
bifurcations_list = None

In [ ]:
cws = eikivp.orientationscore.cakewavelet_stack(dim_I, dim_K, inflection_point=0.8)
U = np.transpose(eikivp.orientationscore.wavelet_transform(u, cws), axes=(1, 2, 0)).real # * dim_I / 2
fig, ax, cbar = eikivp.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max)
fig.colorbar(cbar, ax=ax);

# Compute Vesselness

In [6]:
mask = retinal_array > 0.
mask = sp.ndimage.binary_erosion(mask, iterations=int(np.ceil(σ_s_list.max() * 2)))
mask = np.transpose(np.array([mask for _ in range(dim_K)], dtype=int), axes=(1, 2, 0))

V = multi_scale_vesselness(U, mask, θs, σ_s_list, σ_o, σ_s_ext, σ_o_ext, dxy, dθ, bifurcations=bifurcations_list)
C = cost_function(V, λ, p)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = eikivp.visualisations.plot_image_array(-V.max(-1), x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = eikivp.visualisations.plot_image_array(C.min(-1), x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [8]:
export_vesselness(V, params, "storage\\vesselness")